In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
import pandas as pd
from extract import open_driver, close_driver, wait_for_element
from transforms import parse_record, parse_game_date, game_season
from selenium.webdriver.common.by import By
from DML import db_connect, close_db_connect
from psycopg2.extras import execute_batch
from sql_queries import summary_table_insert
import urllib.parse
from datetime import datetime

In [88]:
def get_table_columns(table_element):
    header_container = wait_for_element(source=table_element, search_by=By.CLASS_NAME, target="table-header-container", unique_element=True)
    header_elements = wait_for_element(source=header_container, search_by=By.CLASS_NAME, target="rt-th")
    header_names = [header.text for header in header_elements]
    
    return header_names

def get_table_rows(table_element):
    table_rows = wait_for_element(source=table_element, search_by=By.CLASS_NAME, target="rt-tbody", unique_element=True)
    rows = wait_for_element(source=table_rows, search_by=By.CLASS_NAME, target="rt-tr")
    
    return rows

In [89]:
class Scraper:
    def __init__(self):
        self.driver = None
        self.conn = None
        self.cur = None
        self.table_data = None
        self.table_headers = None
        self.table_insert = ""
        self.staged_data = []
    
    def open_driver(self):
        self.driver = open_driver()
    
    def close_driver(self):
        if self.driver:
            close_driver(self.driver)
            
    def db_connect(self):
        self.conn, self.cur = db_connect()
    
    def close_db_connect(self):
        if self.conn and self.cur:
            close_db_connect(self.conn, self.cur)
            
    def extract(self, url):
        """sends requests to the url and scrapes the table elements (header and rows)"""
        self.driver.get(url)
        
        # scrape the page's main table
        root_element = wait_for_element(source=self.driver, search_by=By.ID, target="root", unique_element=True)
        
#         page_jump = wait_for_element(source=root_element, search_by=By.CLASS_NAME, target="-pageJump", unique_element=True)
#         page_number = wait_for_element(source=root_element, search_by=By.TAG_NAME, target="input", unique_element=True)
        
        data_table = wait_for_element(source=root_element, search_by=By.CLASS_NAME, target="rt-table", unique_element=True)

        header_elements = get_table_columns(data_table)
        row_elements = get_table_rows(data_table)
        
        return header_elements, row_elements
    

    def load(self, data_rows):
        execute_batch(self.cur, self.table_insert, data_rows)

In [90]:
class SummaryScraper(Scraper):
    def __init__(self):
        super().__init__()
        self.table_insert = summary_table_insert
        self.base_url = "https://www.nhl.com/stats/teams?"
        self.url_dict = {
            "aggregate": 0,
            "reportType": "game",
            "page": 0,
            "pageSize": 100
        }
    
    def build_url(self, start, end=None, playoffs=False):
        if playoffs:
            self.url_dict["gameType"] = 3
        else:
            self.url_dict["gameType"] = 2
            
        if isinstance(start, int):
            self.url_dict["seasonFrom"] = start
            
            if isinstance(end, int):
                self.url_dict["seasonTo"] = end
            else:
                self.url_dict["seasonTo"] = f"{datetime.now().year}{datetime.now().year+1}"
            
            self.url_dict["dateFromSeason"] = []
        
        elif isinstance(start, str) and start.isnumeric():
            self.url_dict["seasonFrom"] = start
            
            if isinstance(end, str) and end.isnumeric():
                self.url_dict["seasonTo"] = end
            else:
                self.url_dict["seasonTo"] = start
            
            self.url_dict["dateFromSeason"] = []
            
        elif isinstance(start, datetime):            
            self.url_dict["dateFrom"] = start.strftime("%Y-%m-%d")
            
            if isinstance(end, datetime):
                self.url_dict["dateTo"] = end.strftime("%Y-%m-%d")
            else:
                self.url_dict["dateTo"] = datetime.now().date()
        
        url = self.base_url + urllib.parse.urlencode(self.url_dict)
        
        return url
        
    def transform(self, headers, row_elements):
        table_data = []
        
        for row in row_elements:
            row_cells = wait_for_element(source=row, search_by=By.CLASS_NAME, target="rt-td")
            row_values = [cell.text for cell in row_cells]
            
            values_map = list(zip(headers, row_values))
            map_dict = parse_record(values_map)
            
            self.cur.execute("SELECT abbreviation FROM teams WHERE team = %s;", (map_dict["Team"],))
            team_abbreviation = self.cur.fetchone()[0]
            
            ordered_data_list = [
                team_abbreviation,
                map_dict["season"],
                map_dict["game_date"],
                map_dict["home_game"],
                map_dict["opponent"],
                map_dict["W"],
                map_dict["L"],
                map_dict["T"],
                map_dict["OT"],
                map_dict["P"],
                map_dict["P%"],
                map_dict["RW"],
                map_dict["ROW"],
                map_dict["S/O Win"],
                map_dict["GF"],
                map_dict["GA"],
                map_dict["PP%"],
                map_dict["PK%"],
                map_dict["Net PP%"],
                map_dict["Net PK%"],
                map_dict["Shots/GP"],
                map_dict["GA/GP"],
                map_dict["FOW%"]
            ]
        
            table_data.append(ordered_data_list)
        
        return table_data
    
    def etl(self, start, playoffs=False):
        # the chrome driver and database cursor are used in multiple scripts
        self.open_driver()
        self.db_connect()
        
        url = self.build_url(start=start)
        table_headers, table_rows = self.extract(url)

        parsed_data = self.transform(table_headers, table_rows)        
        
#         execute batch loading
#         self.load(parsed_data)
        
        # close the driver and the cursor
        self.close_driver()
        self.close_db_connect()
        
        return parsed_data

In [91]:
scraper = SummaryScraper()

In [92]:
summary_table_data = scraper.etl(start=2020)

In [85]:
summary_table_data

[['NYR',
  '2021/22',
  datetime.date(2022, 3, 12),
  False,
  'DAL',
  1,
  0,
  None,
  0,
  2,
  1.0,
  1,
  1,
  0,
  7,
  4,
  50.0,
  50.0,
  50.0,
  50.0,
  29.0,
  4.0,
  37.3],
 ['NYR',
  '2021/22',
  datetime.date(2022, 4, 9),
  True,
  'OTT',
  1,
  0,
  None,
  0,
  2,
  1.0,
  1,
  1,
  0,
  5,
  1,
  0.0,
  100.0,
  0.0,
  100.0,
  33.0,
  1.0,
  52.6],
 ['NYR',
  '2021/22',
  datetime.date(2022, 3, 15),
  True,
  'ANA',
  1,
  0,
  None,
  0,
  2,
  1.0,
  0,
  1,
  0,
  4,
  3,
  50.0,
  66.7,
  50.0,
  66.7,
  38.0,
  3.0,
  51.9],
 ['NYR',
  '2021/22',
  datetime.date(2022, 4, 16),
  True,
  'DET',
  1,
  0,
  None,
  0,
  2,
  1.0,
  1,
  1,
  0,
  4,
  0,
  25.0,
  100.0,
  25.0,
  100.0,
  37.0,
  0.0,
  60.0],
 ['NYR',
  '2021/22',
  datetime.date(2022, 3, 29),
  False,
  'PIT',
  1,
  0,
  None,
  0,
  2,
  1.0,
  1,
  1,
  0,
  3,
  2,
  100.0,
  66.7,
  100.0,
  66.7,
  26.0,
  2.0,
  45.8],
 ['NYR',
  '2021/22',
  datetime.date(2022, 2, 15),
  True,
  'BOS',
 